import libraries

In [1]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import BasePreprocessor
from mltrainer import imagemodels, Trainer, TrainerSettings, ReportTypes, metrics
import torch.optim as optim
from torch import nn
from tomlserializer import TOMLSerializer

get dataset

In [2]:
fashionfactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
preprocessor = BasePreprocessor()
streamers = fashionfactory.create_datastreamer(batchsize=64, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]
trainstreamer = train.stream()
validstreamer = valid.stream()

2025-10-15 20:55:34.022 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at C:\Users\tycoh\.cache\mads_datasets\fashionmnist
2025-10-15 20:55:34.024 | INFO     | mads_datasets.base:download_data:124 - File already exists at C:\Users\tycoh\.cache\mads_datasets\fashionmnist\fashionmnist.pt


set metric and loss function

In [3]:
import torch
accuracy = metrics.Accuracy()
loss_fn = torch.nn.CrossEntropyLoss()

create simpel model

In [21]:
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes: int, units1: int, units2: int, units3: int, units4: int) -> None:
        super().__init__()
        self.num_classes = num_classes
        self.units1 = units1
        self.units2 = units2
        self.units3 = units3
        self.units4 = units4
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, units1),
            nn.ReLU(),
            nn.Linear(units1, units2),
            nn.ReLU(),
            nn.Linear(units2, units3),
            nn.ReLU(),
            nn.Linear(units3, units4),
            nn.ReLU(),
            nn.Linear(units4, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

set model settings (this can be changed for the experiment)

In [33]:
settings = TrainerSettings(
    epochs=10,
    metrics=[accuracy],
    logdir="modellogs",
    train_steps= len(train),
    valid_steps= len(valid),
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
)

train model and save settings

In [34]:
units = 128
model = NeuralNetwork(num_classes=10, units1=units, units2=units, units3=units, units4=units)

trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau
)
trainer.loop()

tomlserializer = TOMLSerializer()
tomlserializer.save(settings, "settings.toml")
tomlserializer.save(model, "model.toml")

2025-10-15 21:12:46.563 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to modellogs\20251015-211246
2025-10-15 21:12:46.565 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 937/937 [00:04<00:00, 205.85it/s]
2025-10-15 21:12:51.442 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 0.5916 test 0.4824 metric ['0.8235']
100%|██████████| 937/937 [00:05<00:00, 159.01it/s]
2025-10-15 21:12:57.691 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.3948 test 0.4031 metric ['0.8555']
100%|██████████| 937/937 [00:09<00:00, 102.37it/s]
2025-10-15 21:13:07.726 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.3505 test 0.3807 metric ['0.8619']
100%|██████████| 937/937 [00:08<00:00, 108.23it/s]
2025-10-15 21:13:16.993 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 0.3257 test 0.3677 metric ['0.8692']
100%|██████████| 937/937 [00:10<00:00, 89.35it/s] 
2025-1